In [ ]:
CREATE TABLE IF NOT EXISTS DATA_SLV.DIM_TOPICS (
    TOPIC_ID NUMBER(38,0),
    MAIN_TOPIC_ID NUMBER(38,0),
    DESCRIPTION VARCHAR,
    TOPIC_NAME VARCHAR,
    URLKEY VARCHAR,
    MEMBERS NUMBER(38,0),
    LINK VARCHAR
);

MERGE INTO DATA_SLV.DIM_TOPICS AS d
USING (
  SELECT DISTINCT
    TRY_TO_NUMBER(t.TOPIC_ID)          AS topic_id,        -- NK numérica
    TRY_TO_NUMBER(t.MAIN_TOPIC_ID)     AS main_topic_id,   -- puede ser NULL
    NULLIF(TRIM(t.DESCRIPTION), '')    AS description,
    NULLIF(TRIM(t.TOPIC_NAME),  '')    AS topic_name,
    NULLIF(TRIM(t.URLKEY),      '')    AS urlkey,
    TRY_TO_NUMBER(t.MEMBERS)           AS members,
    NULLIF(TRIM(t.LINK),        '')    AS link
  FROM MEETUP_DB.DATA_BRZ.TOPICS t
  WHERE TRY_TO_NUMBER(t.TOPIC_ID) IS NOT NULL
) AS s
ON d.TOPIC_ID = s.topic_id

WHEN MATCHED THEN UPDATE SET
  d.MAIN_TOPIC_ID = s.main_topic_id,
  d.DESCRIPTION   = s.description,
  d.TOPIC_NAME    = s.topic_name,
  d.URLKEY        = s.urlkey,
  d.MEMBERS       = s.members,
  d.LINK          = s.link

WHEN NOT MATCHED THEN INSERT (
  TOPIC_ID, MAIN_TOPIC_ID, DESCRIPTION, TOPIC_NAME, URLKEY, MEMBERS, LINK
) VALUES (
  s.topic_id, s.main_topic_id, s.description, s.topic_name, s.urlkey, s.members, s.link
);